In [1]:
import json
import re
import os

import time 
import ssl
from urllib.request import Request, urlopen

import requests
from lxml.html import fromstring


from bs4 import BeautifulSoup as bs
BASE_URL = "https://www.indeed.com/"
BROWSER = 'Mozilla/89.0.1'

In [2]:
#ignore ssl certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

dir_path = os.path.dirname(os.path.realpath("__file__"))

#### Get Proxies

In [3]:
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:1000]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return list(proxies)
proxies = get_proxies()

In [4]:
from datetime import datetime
date = datetime.today().strftime('%Y_%m_%d')

In [5]:
CITIES = [
    "New York",
    "Chicago",
    "Boston",
    "Washington DC",
    "Seattle",
    
    "Denver",
    "San Francisco",
]
KEYWORDS = "data science"

In [6]:
def main(CITY, FILENAME, KEYWORDS):
    name_of_city = CITY
    keywords = KEYWORDS
    number_of_pages = 20

    #Preprocess data
    name_of_city = name_of_city.replace(" ", "+")
    keywords = keywords.replace(" ", "+")
    number_of_pages = int(number_of_pages)

    url_to_scrape = BASE_URL + "/jobs?q=" + keywords + "&l=" + name_of_city
    data_collected = scrape_data(url_to_scrape, number_of_pages)

    with open(dir_path + '/' + FILENAME, "w") as file:
        json.dump(data_collected, file, sort_keys=True, indent = 4, ensure_ascii=False)

In [7]:
def scrape_data(url, number_of_pages):
    data_collected = []
    
    PROXY_COUNT = 15
    for i in range(number_of_pages):
        proxy = proxies[PROXY_COUNT]
        PROXY_COUNT += 1
        if PROXY_COUNT >= len(proxies):
            PROXY_COUNT= 0 
        extension = ""
        if i is not 0:
            extension = "&start=" + str(i*10)
            url_added = url + extension
            print(url_added)
            
            pass_crit = True
            web_page = ""
            temp_count = 0
            while(pass_crit or temp_count ==len(proxies) + 5):
                temp_count +=1
                try:
                    web_page = requests.get(url_added,proxies={"http": proxy, "https": proxy})
                    pass_crit = False
                except:
                    proxy = proxies[PROXY_COUNT]
                    PROXY_COUNT += 1
                    if PROXY_COUNT >= len(proxies):
                        PROXY_COUNT= 0 
                    
            data_raw = bs(web_page.text, 'html.parser')
            data_collected = get_data_html(data_collected, data_raw, PROXY_COUNT)
            time.sleep(.5)
            with open(dir_path + '/' + FILENAME, "w") as file:
                json.dump(data_collected, file, sort_keys=True, indent = 4, ensure_ascii=False)

    return data_collected


def get_data_html(data_collected, data_raw, PROXY_COUNT):
    job_posts = []
    for div in data_raw.findAll('div', attrs={'class':'jobsearch-SerpJobCard unifiedRow row result'}):
        job = dict()
        job = extract_data_points(job, div)
        job_posts.append(div['data-jk'])
        single_job_post_extension_url = "https://www.indeed.com/viewjob?jk=" + div['data-jk']
        job['url'] = single_job_post_extension_url
        
        pass_crit = True
        web_page = ""
        temp_count= 0
        while(pass_crit or temp_count ==len(proxies) + 5):
            temp_count +=1
            try:
                web_page = requests.get(single_job_post_extension_url,proxies={"http": proxy, "https": proxy})
                pass_crit = False
            except:
                proxy = proxies[PROXY_COUNT]
                PROXY_COUNT += 1
                if PROXY_COUNT >= len(proxies):
                    PROXY_COUNT= 0 
        
        job_raw = bs(web_page.text, 'html.parser')
        for inside_div in job_raw.findAll('div', attrs={"class": "jobsearch-jobDescriptionText"}):
            
            details = inside_div.text.strip()
            job["details"] = details.replace(" +", "")
        data_collected.append(job)
    return data_collected


def extract_data_points(job, div):
    for a in div.findAll('a', attrs={"class": "jobtitle turnstileLink"}):
        job['title'] = a['title']
    for a1 in div.findAll('a', attrs={"data-tn-element": "companyName"}):
        job['companyName'] = a1.text.strip()
    for span in div.findAll('span', attrs={"class": "ratingsContent"}):
        job['rating'] = span.text.strip()
    for span1 in div.findAll('span', attrs={"class": "location accessible-contrast-color-location"}):
        job['location'] = span1.text.strip()
    for div1 in div.findAll('div', attrs={"class": "summary"}):
        summary = div1.text.strip()
        job['summary'] = summary.replace(' +', "")
    for span2 in div.findAll('span', attrs={'class':'date'}):
        job['date'] = span2.text.strip()
    job['date_queried'] = datetime.today().strftime('%Y-%m-%d')
    return job

In [9]:
from multiprocessing import Process

for CITY in CITIES:
    city = CITY.replace(" ", "_")
    FILENAME = f"data_{date}_{city}.json"

    p = Process(target=main, args=(CITY, FILENAME, KEYWORDS))
    p.start()

https://www.indeed.com//jobs?q=data+science&l=New+York&start=10https://www.indeed.com//jobs?q=data+science&l=Chicago&start=10https://www.indeed.com//jobs?q=data+science&l=Boston&start=10

https://www.indeed.com//jobs?q=data+science&l=Washington+DC&start=10
https://www.indeed.com//jobs?q=data+science&l=Seattle&start=10https://www.indeed.com//jobs?q=data+science&l=Denver&start=10


https://www.indeed.com//jobs?q=data+science&l=San+Francisco&start=10
https://www.indeed.com//jobs?q=data+science&l=New+York&start=20
https://www.indeed.com//jobs?q=data+science&l=New+York&start=30
https://www.indeed.com//jobs?q=data+science&l=Boston&start=20
https://www.indeed.com//jobs?q=data+science&l=Washington+DC&start=20
https://www.indeed.com//jobs?q=data+science&l=Boston&start=30
https://www.indeed.com//jobs?q=data+science&l=Boston&start=40
https://www.indeed.com//jobs?q=data+science&l=Chicago&start=20
https://www.indeed.com//jobs?q=data+science&l=Seattle&start=20
https://www.indeed.com//jobs?q=data+sci